# Imports & Config

In [420]:
import os
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from plotly.subplots import make_subplots
from dotenv import dotenv_values

config = dotenv_values(".env")

px.set_mapbox_access_token(config['MAPBOX_TOKEN'])
pio.templates.default = "plotly_white"

pd.options.display.max_rows = 85

In [421]:
df = pd.read_csv('train.csv')

# Todos & Ideas

Feature Engineering:
- Col for # of stories, e.g. 2-story house (see col `MSSubClass`)
- Further breakout `MSSubClass`
- Exterior features, like type, quality and condition can liked be combined.
- There are a lot of basement features that can likely be combined
- Utilitites, like heating, central air, electrical type may be able to be consolidated. Should check if there is high correlation. Also, year built/renovated likely plays role here too
- `BsmtFullBath` and `BsmtHalfBath` can likely be combined. Potentially they can be added to total bathrooms too.
- `FullBath` & `HalfBath` can be combined
- `GarageArea` & `GarageCars` consider new col which would be area per car.
- Consolidate Garage features, such as `GarageCond` & `GarageQual`
- Porch types might be able to be consolidated into 2 cols: porch type + size
- `MoSold` & `YearSold` and price are likely large predictors
- Convert MoSold and YrSold into a single integer col, for instance, yr 2 mo 3 would be 15

Feature Adjustment:
- All categorical values need to be encoded
- All continuous features should be two-tailed trimmed for extreme values.
- `LotFrontage` 18% NA values.
- `LotFrontage` has an extreme value
- `LotArea` has extreme values an a slight right skew
- `LandContour` single value, "Lvl" makes up 90% of records. Consider switching to bool or removing
- `LandSlope` similar to LandContour in meaning and in value distribution
- `Condition1` 86% single value, 9 potential options. Should consider converting to bool or grouping less prevalent cols together
- `BldgType` may be able to combine with other construction type features, such as n_stories
- `HouseStyle` similar to `BldgType`, might be able to be combined
- `YearBuilt` has 3 peaks and skews heavy left. Should consider normalizing
- `YearRemodAdd` if no remodel, value is equal to `YearBuilt`. Should consider making value NA instead
- `RoofStyle` consider grouping least common 4 into an "other" bin
- `Exterior1st` consider grouping least common into an "other" bin
- `Exterior2nd` instead of NA, if house only has single exterior type, value is duplicated here. Should consider switching to NA and/or grouping with `Exterior1st`
- `MasVnrType` has 8 NAs. There is a string value for "None" and these 8 should be combined with "None"
- `MasVnrArea` mostly 0's. Otherwise, long right tail. should consider logging.
- `ExterQual` & `ExterCond` should be converted to 1-5 (or 1-10) rating. Potentially groouped with other exterior features.
- `BsmtQual` is a rating based on height of basement... should be renamed to BsmtHeght and made on scale 1 - 10. Potentially should be combined with `BsmtCond` on similarly remapped scale
- `BsmtFinType1` and 2 - could show partially completed basements? Likely all basement features should be consolidated.
- `BsmtUnfSF` skews right, of course
- `2ndFlrSF` uses 0 values to indicate no sqft, which actually is probably fine
- `Functional` 93% are single value. Consider making binary
- `GarageType` and other garage features: 81 NAs. This is fine, b ut when converting to categorical, make 0
- Porch/Deck features have right skewed values (and signficant zeros)
- `Fence` - 80% of values are NA
- `MiscFeature` 94% NA.. but could be big hitters, e.g. an elevator!!!
- `MiscVal` should potentially be subtracted from sale price
- `SaleType` several options are sparse nd we should consider consolidating into an "other" value

Feature Removal:
- `Street` has two options, Grvl only has 6 out of 1460 records
- `Alley` has a lot of NA values (93.77%)
- `Utilities` 99% records have value "AllPub"
- `Condition2` 98.97% records have "Norm" value.
- `PoolQC` 99.52% NA. consider grouping?
- `MiscFeature` 96.3% NA. consider grouping.
- `RoofMatl` 98.2% single value
- `Heating` 98% single value
- `LowQualFinSF` 98.2% have 0 value
- `PoolArea` 0.48% of records have a pool

Misc:
- I bet there is a strong correlation between building type and year built, as well as overall quality
- Exterior Quality, Exterior Condition, Exterior Type, these are likely all correlated. Should foundation and rooftype be included? How about building type?
- 32.4% of basements are fully completed
- A wood deck is not exclusive with the other porch types


# Individual Column Analysis

## Helpers

In [422]:
# A common value for many of the columns
qualitative_rating_map = {
    "Ex": "Excellent",
    "Gd": "Good",
    "TA": "Average",
    "Fa": "Fair",
    "Po": "Poor",
}

## MSSubClass

*Property Type*

In [423]:
df.MSSubClass.isna().sum()

0

In [424]:
property_subtype_mapper = {
    20: "1-STORY 1946 & NEWER ALL STYLES",
    30: "1-STORY 1945 & OLDER",
    40: "1-STORY W/FINISHED ATTIC ALL AGES",
    45: "1-1/2 STORY - UNFINISHED ALL AGES",
    50: "1-1/2 STORY FINISHED ALL AGES",
    60: "2-STORY 1946 & NEWER",
    70: "2-STORY 1945 & OLDER",
    75: "2-1/2 STORY ALL AGES",
    80: "SPLIT OR MULTI-LEVEL",
    85: "SPLIT FOYER",
    90: "DUPLEX - ALL STYLES AND AGES",
    120: "1-STORY PUD (Planned Unit Development) - 1946 & NEWER",
    150: "1-1/2 STORY PUD - ALL AGES", # doesn't show up in data
    160: "2-STORY PUD - 1946 & NEWER",
    180: "PUD - MULTILEVEL - INCL SPLIT LEV/FOYER",
    190: "2 FAMILY CONVERSION - ALL STYLES AND AGES",
}

In [425]:
fig = px.bar(df.MSSubClass.replace(property_subtype_mapper).value_counts())
fig.update_xaxes(tickangle=45)
fig.show()

## MSZoning

*Zoning Classification*

In [426]:
df.MSZoning.isna().sum()

0

In [427]:
col_map = {
    "A" : "Agriculture",
    "C" : "Commercial",
    "FV": "Floating Village Residential",
    "I" : "Industrial",
    "RH": "Residential High Density",
    "RL": "Residential Low Density",
    "RP": "Residential Low Density Park ",
    "RM": "Residential Medium Density",
}

In [428]:
px.bar(df.MSZoning.replace(col_map).value_counts())

## LotFrontage

*Linear Feet of street connected to property*

In [429]:
df.LotFrontage.isna().sum()

259

In [430]:
df.LotFrontage.describe()

count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

In [431]:
px.histogram(df.LotFrontage)

## LotArea

*Lot size in sqft*

In [432]:
df.LotArea.isna().sum()

0

In [433]:
df.LotArea.describe()

count      1460.000000
mean      10516.828082
std        9981.264932
min        1300.000000
25%        7553.500000
50%        9478.500000
75%       11601.500000
max      215245.000000
Name: LotArea, dtype: float64

In [434]:
px.histogram(df.LotArea)

## Street

*Type of Road Access*

In [435]:
df.Street.isna().sum()

0

In [436]:
df.Street.value_counts()

Pave    1454
Grvl       6
Name: Street, dtype: int64

In [653]:
px.histogram(df, x="SalePrice", color="Street", marginal="violin")

## Alley

*Type of alley access*

In [437]:
df.Alley.isna().sum() / len(df)

0.9376712328767123

In [438]:
df.Alley.value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [657]:
px.histogram(df.fillna("None"), x="SalePrice", color="Alley", marginal="violin")

## LotShape

*General Shape of Property*

In [439]:
df.LotShape.isna().sum()

0

In [440]:
col_map = {
    'Reg': 'Regular	',
    'IR1': 'Slightly irregular',
    'IR2': 'Moderately Irregular',
    'IR3': 'Irregular',
}

In [441]:
px.bar(df.LotShape.replace(col_map).value_counts())

## LandContour

*Flatness of property*

In [442]:
df.LandContour.isna().sum()

0

In [443]:
df.LandContour.value_counts()

Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64

## Utilites

*Availble Utilities*

In [444]:
df.Utilities.isna().sum()

0

In [445]:
df.Utilities.value_counts()

AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64

## LotConfig

*Lot configuration*

- Inside  -> Inside lot
- Corner  -> Corner lot
- CulDSac -> Cul-de-sac
- FR2     -> Frontage on 2 sides of property
- FR3     -> Frontage on 3 sides of property

In [446]:
df.LotConfig.isna().sum()

0

In [447]:
px.bar(df.LotConfig.value_counts())

## LandSlope

*Slope of Property*

Different than LandContour?

In [448]:
df.LandSlope.isna().sum()

0

In [449]:
df.LandSlope.value_counts()

Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64

## Neighborhood

*You know, 'hoods*

In [450]:
df.Neighborhood.isna().sum()

0

In [451]:
col_map = {
    'Blmngtn': "Bloomington Heights",
    'Blueste': "Bluestem",
    'BrDale': "Briardale",
    'BrkSide': "Brookside",
    'ClearCr': "Clear Creek",
    'CollgCr': "College Creek",
    'Crawfor': "Crawford",
    'Edwards': "Edwards",
    'Gilbert': "Gilbert",
    'IDOTRR': "Iowa DOT and Rail Road",
    'MeadowV': "Meadow Village",
    'Mitchel': "Mitchell",
    'NAmes': "North Ames",
    'NoRidge': "Northridge",
    'NPkVill': "Northpark Villa",
    'NridgHt': "Northridge Heights",
    'NWAmes': "Northwest Ames",
    'OldTown': "Old Town",
    'SWISU': "South & West of Iowa State University",
    'Sawyer': "Sawyer",
    'SawyerW': "Sawyer West",
    'Somerst': "Somerset",
    'StoneBr': "Stone Brook",
    'Timber': "Timberland",
    'Veenker': "Veenker",
}

In [452]:
fig = px.bar(df.Neighborhood.replace(col_map).value_counts())
fig.update_xaxes(tickangle=45)
fig.show()

In [453]:
coord_map = {
    'Blmngtn': (42.0615560174058, -93.63632087007232),
    'Blueste': (42.00980422918034, -93.64713037086807),
    'BrDale' : (42.05294634177981, -93.62877808673957),
    'BrkSide': (42.03261807330741, -93.6238377290682),
    'ClearCr': (42.060171214588586, -93.62907296875137),
    'CollgCr': (42.02129647632772, -93.68298605975636),
    'Crawfor': (42.01845910953693, -93.64882844665684),
    'Edwards': (42.02544200932463, -93.66961235790437),
    'Gilbert': (42.0601393017811, -93.64178532245417),
    'IDOTRR' : (42.021926028596006, -93.6225329652928),
    'MeadowV': (41.99191886480588, -93.6037245867413),
    'Mitchel': (41.99139287383528, -93.60083872536556),
    'NAmes'  : (42.04419386800324, -93.61902560393564),
    'NoRidge': (42.052681382205016, -93.65258817721676),
    'NPkVill': (42.05032314537289, -93.62692345975557),
    'NridgHt': (42.0591675764353, -93.65650611425234),
    'NWAmes' : (42.04850339629937, -93.63488406954389),
    'OldTown': (42.03071605999881, -93.61563439838042),
    'SWISU'  : (42.02229601189108, -93.65646834813718),
    'Sawyer' : (42.03300789288219, -93.66937687880377),
    'SawyerW': (42.032585598131774, -93.68262437880381),
    'Somerst': (42.05116846741627, -93.64484313846206),
    'StoneBr': (42.0580947108055, -93.63327744441142),
    'Timber' : (41.99898805473445, -93.6488273057909),
    'Veenker': (42.04233297919965, -93.64902544255996),
}

In [454]:
names, lats, lngs = [],[],[]
for name, (lat, lng) in coord_map.items():
    names.append(name)
    lats.append(lat)
    lngs.append(lng)

px.scatter_mapbox(lat=lats, lon=lngs, color=names, text=names, height=800)

## Condition1

*Proximity to the following conditions*

- Norm -> Normal
- Feedr -> Adjacent to feeder street	
- Artery -> Adjacent to arterial street
- RRAn -> Adjacent to North-South Railroad
- PosN -> Near positive off-site feature--park, greenbelt, etc.
- RRAe -> Adjacent to East-West Railroad
- PosA -> Adjacent to postive off-site feature
- RRNn -> Within 200' of North-South Railroad
- RRNe -> Within 200' of East-West Railroad

In [455]:
df.Condition1.isna().sum()

0

In [456]:
px.bar(df.Condition1.value_counts())

In [457]:
len(df.Condition1[df.Condition1=="Norm"]) / len(df)

0.863013698630137

## Condition2

*Same as Condition1, value is present if property a second applicable condition*

In [458]:
df.Condition2.isna().sum()

0

In [459]:
df.Condition2.value_counts()

Norm      1445
Feedr        6
Artery       2
RRNn         2
PosN         2
PosA         1
RRAe         1
RRAn         1
Name: Condition2, dtype: int64

In [460]:
len(df.Condition2[df.Condition2=='Norm']) / len(df)

0.9897260273972602

## BldgType

*House build type*

In [461]:
df.BldgType.isna().sum()

0

In [462]:
col_map = {
    "1Fam": "Single-family Detached",
    "2FmCon": "Two-family Conversion; originally built as one-family dwelling",
    "Duplx": "Duplex",
    "TwnhsE": "Townhouse End Unit",
    "TwnhsI": "Townhouse Inside Unit",
}

In [463]:
px.bar(df.BldgType.replace(col_map).value_counts())

## HouseStyle

*Style of House*

In [464]:
col_map = {
    "1Story": "One story",
    "1.5Fin": "One and one-half story: 2nd level finished",
    "1.5Unf": "One and one-half story: 2nd level unfinished",
    "2Story": "Two story",
    "2.5Fin": "Two and one-half story: 2nd level finished",
    "2.5Unf": "Two and one-half story: 2nd level unfinished",
    "SFoyer": "Split Foyer",
    "SLvl": "Split Level",
}

In [465]:
px.bar(df.HouseStyle.replace(col_map).value_counts())

## OverallQual

*Rates from 1-10 the overall material and finish of the house*

In [466]:
df.OverallQual.isna().sum()

0

In [467]:
px.bar(df.OverallQual.value_counts())

## OverallCond

*1-10 rating of condition*

Ha - there are no 10 ratings, but also only 1 "1" rating

In [468]:
df.OverallCond.isna().sum()

0

In [469]:
df.OverallCond.dtype

dtype('int64')

In [470]:
px.bar(df.OverallCond.value_counts())

## YearBuilt

*Original date of construction*

In [471]:
df.YearBuilt.isna().sum()

0

In [472]:
df.YearBuilt.dtype

dtype('int64')

In [473]:
df.YearBuilt.describe()

count    1460.000000
mean     1971.267808
std        30.202904
min      1872.000000
25%      1954.000000
50%      1973.000000
75%      2000.000000
max      2010.000000
Name: YearBuilt, dtype: float64

In [474]:
px.histogram(df.YearBuilt)

## YearRemodAdd

*Remodel Date*

Note: If no remodel date, will equal `YearBuilt` instead of NA

In [475]:
df.YearRemodAdd.isna().sum()

0

In [476]:
built_vs_model = df.loc[:,['YearBuilt', 'YearRemodAdd']]

built_vs_model['label_1'] = 'Year Built'
built_vs_model['label_2'] = 'Year Renovated'

built_vs_model = pd.concat([
    built_vs_model[['YearBuilt', 'label_1']].rename(columns={'YearBuilt':'year', 'label_1':'label'}), 
    built_vs_model[['YearRemodAdd', 'label_2']].rename(columns={'YearRemodAdd':'year', 'label_2':'label'})
])

In [477]:
px.histogram(built_vs_model, x="year", color="label", barmode="overlay", title='Year Built vs Renovated')

## RoofStyle

In [478]:
df.RoofStyle.isna().sum()

0

In [479]:
px.bar(df.RoofStyle.value_counts())

## RoofMatl

*Roof material*

In [480]:
df.RoofMatl.isna().sum()

0

In [481]:
col_map = {
    "ClyTile": "Clay or Tile",
    "CompShg": "Standard (Composite) Shingle",
    "Membran": "Membrane",
    "Metal": "Metal",
    "Roll": "Roll",
    "Tar&Grv": "Gravel & Tar",
    "WdShake": "Wood Shakes",
    "WdShngl": "Wood Shingles",
}

In [482]:
px.bar(df.RoofMatl.replace(col_map).value_counts())

In [483]:
len(df.RoofMatl[df.RoofMatl == 'CompShg']) / len(df)

0.9821917808219178

## Exterior1st

*Type of exterior wall*

In [484]:
df.Exterior1st.isna().sum()

0

In [485]:
col_map = {
    "AsbShng":	"Asbestos Shingles",
    "AsphShn":	"Asphalt Shingles",
    "BrkComm":	"Brick Common",
    "BrkFace":	"Brick Face",
    "CBlock":	"Cinder Block",
    "CemntBd":	"Cement Board",
    "HdBoard":	"Hard Board",
    "ImStucc":	"Imitation Stucco",
    "MetalSd":	"Metal Siding",
    "Other":	"Other",
    "Plywood":	"Plywood",
    "PreCast":	"PreCast",
    "Stone":	"Stone",
    "Stucco":	"Stucco",
    "VinylSd":	"Vinyl Siding",
    "Wd Sdng":	"Wood Siding",
    "WdShing":	"Wood Shingles",
}

In [486]:
px.bar(df.Exterior1st.replace(col_map).value_counts())

## Exterior2nd

*Same as `Exterior1st`. Value here indicates a second type of exterior material*

In [487]:
df.Exterior2nd.isna().sum()

0

In [488]:
col_map = {
    "AsbShng":	"Asbestos Shingles",
    "AsphShn":	"Asphalt Shingles",
    "BrkComm":	"Brick Common",
    "BrkFace":	"Brick Face",
    "CBlock":	"Cinder Block",
    "CemntBd":	"Cement Board",
    "HdBoard":	"Hard Board",
    "ImStucc":	"Imitation Stucco",
    "MetalSd":	"Metal Siding",
    "Other":	"Other",
    "Plywood":	"Plywood",
    "PreCast":	"PreCast",
    "Stone":	"Stone",
    "Stucco":	"Stucco",
    "VinylSd":	"Vinyl Siding",
    "Wd Sdng":	"Wood Siding",
    "WdShing":	"Wood Shingles",
}

In [489]:
px.bar(df.Exterior2nd.replace(col_map).value_counts())

In [490]:
(df.Exterior1st == df.Exterior2nd).sum()

1245

In [491]:
(df.Exterior1st != df.Exterior2nd).sum()

215

Could be the case where if there is only one material, its value is repeated in this column instead of a NA.

## MasVnrType

*Masonry veneer type*

In [492]:
df.MasVnrType.isna().sum()

8

In [493]:
col_map = {
    "BrkCmn": "Brick Common",
    "BrkFace": "Brick Face",
    "CBlock": "Cinder Block",
    "None": "None",
    "Stone": "Stone",
}

In [494]:
px.bar(df.MasVnrType.replace(col_map).value_counts())

## MasVnrArea

*Masonry Veneer area in sqft*

In [495]:
px.histogram(df.MasVnrArea)

0 values here overlap well with the "None" value in `MasVnrType`. Oh good.

## ExterQual

*Quality of exterior material*
Note: scale is similar to `ExterCond`.. wonder how these two interplay, as well as how they correlate with `Exterior1st`

In [496]:
df.ExterQual.isna().sum()

0

In [497]:
col_map = {
    "Ex":	"Excellent",
    "Gd":	"Good",
    "TA":	"Average/Typical",
    "Fa":	"Fair",
    "Po":	"Poor",
}

In [498]:
px.bar(df.ExterQual.replace(col_map).value_counts())

## ExterCond

*Similar to above col `ExterQual`.*

In [499]:
df.ExterCond.isna().sum()

0

In [500]:
col_map = {
    "Ex":	"Excellent",
    "Gd":	"Good",
    "TA":	"Average/Typical",
    "Fa":	"Fair",
    "Po":	"Poor",
}

In [501]:
px.bar(df.ExterCond.replace(col_map).value_counts())

## Foundation

*Foundation Material*

In [502]:
df.Foundation.isna().sum()

0

In [503]:
col_map = {
    "BrkTil": "Brick & Tile",
    "CBlock": "Cinder Block",
    "PConc": "Poured Contrete",
    "Slab": "Slab",
    "Stone": "Stone",
}

In [504]:
px.bar(df.Foundation.replace(col_map).value_counts())

## BsmtQual

*Quality here just measures height of basement. Well that is misleading*

- Ex -> Excellent (100+ inches)
- Gd -> Good (90-99 inches)
- TA -> Typical (80-89 inches)
- Fa -> Fair (70-79 inches)
- Po -> Poor (<70 inches
- NA -> No Basement

In [505]:
df.BsmtQual.isna().sum()

37

In [506]:
col_map = {
    "Ex": "Excellent (100+ inches)",
    "Gd": "Good (90-99 inches)",
    "TA": "Typical (80-89 inches)",
    "Fa": "Fair (70-79 inches)",
    "Po": "Poor (<70 inches",
    "NA": "No Basement",
}

In [507]:
px.bar(df.BsmtQual.replace(col_map).value_counts())

## BsmtCond

*Condition of Basement*

In [508]:
df.BsmtCond.isna().sum()

37

In [509]:
col_map = {
    "Ex": "Excellent",
    "Gd": "Good",
    "TA": "Typical - slight dampness allowed",
    "Fa": "Fair - dampness or some cracking or settling",
    "Po": "Poor - Severe cracking, settling, or wetness",
    "NA": "No Basement",
}

In [510]:
px.bar(df.BsmtCond.replace(col_map).value_counts())

## BsmtExposure

*Refers to walkout or garden level walls*

In [511]:
df.BsmtExposure.isna().sum()

38

In [512]:
col_map = {
    "Gd": "Good Exposure",
    "Av": "Average Exposure (split levels or foyers typically score average or above)",
    "Mn": "Mimimum Exposure",
    "No": "No Exposure",
    "NA": "No Basement",
}

In [513]:
px.bar(df.BsmtExposure.replace(col_map).value_counts())

## BsmtFinType1

*Rating of basement finished area*

In [514]:
df.BsmtFinType1.isna().sum()

37

In [515]:
col_map = {
    "GLQ": "Good Living Quarters",
    "ALQ": "Average Living Quarters",
    "BLQ": "Below Average Living Quarters",
    "Rec": "Average Rec Room",
    "LwQ": "Low Quality",
    "Unf": "Unfinshed",
    "NA": "No Basement",
}

In [516]:
px.bar(df.BsmtFinType1.replace(col_map).value_counts())

## BsmtFinSF1

*If basement has a finished area, sqft of it*

In [517]:
px.histogram(df.BsmtFinSF1)

## BsmtFinType2

*If there is a second type of basement finish, rating of 2nd basement finished area*

In [518]:
df.BsmtFinType2.isna().sum()

38

In [519]:
col_map = {
    "GLQ": "Good Living Quarters",
    "ALQ": "Average Living Quarters",
    "BLQ": "Below Average Living Quarters",
    "Rec": "Average Rec Room",
    "LwQ": "Low Quality",
    "Unf": "Unfinshed",
    "NA": "No Basement",
}

In [520]:
px.bar(df.BsmtFinType2.replace(col_map).value_counts())

In [521]:
(df.BsmtFinType1 == df.BsmtFinType2).sum()

431

I wonder if a `BsmtFinType1` with a value combined with a `BsmtFinType2` means a partially finished basement. Likely will see from `BsmtUnfSF` vs `TotalBsmtSF`

## BsmtFinSF2

*If basement has a second finished area, sqft of it*

In [522]:
px.histogram(df.BsmtFinSF2)

## BsmtFullBath

*Count of full bathrooms in basement*

In [523]:
df.BsmtFullBath.isna().sum()

0

In [524]:
px.bar(df.BsmtFullBath.value_counts())

## BsmtHalfBath 

*Count of half bathrooms in basement*

In [525]:
df.BsmtHalfBath.isna().sum()

0

In [526]:
px.bar(df.BsmtHalfBath.value_counts())

## BsmtUnfSF

*Total unfinished basement sqft*

In [527]:
df.BsmtUnfSF.isna().sum()

0

In [528]:
df.BsmtUnfSF.describe()

count    1460.000000
mean      567.240411
std       441.866955
min         0.000000
25%       223.000000
50%       477.500000
75%       808.000000
max      2336.000000
Name: BsmtUnfSF, dtype: float64

In [529]:
px.histogram(df.BsmtUnfSF)

## TotlBsmtSF

*Total Basement sqft, finished and unfinished*

In [530]:
px.histogram(df.TotalBsmtSF)

In [531]:
px.histogram(
    df.BsmtUnfSF[(df.BsmtUnfSF > 0) & (df.TotalBsmtSF > 0)] / df.TotalBsmtSF[(df.BsmtUnfSF > 0) & (df.TotalBsmtSF > 0)],
    title="% of basement completion"
)

In [532]:
# % of basements fully completed
461 / len(df.TotalBsmtSF[df.TotalBsmtSF > 0])

0.3239634574841883

## Heating

*Type of Heating*

In [533]:
df.Heating.isna().sum()

0

- Floor -> Floor Furnace
- GasA -> Gas forced warm air furnace
- GasW -> Gas hot water or steam heat
- Grav -> Gravity furnace
- OthW -> Hot water or steam heat other than gas
- Wall -> Wall furnace

In [534]:
df.Heating.value_counts()

GasA     1428
GasW       18
Grav        7
Wall        4
OthW        2
Floor       1
Name: Heating, dtype: int64

In [535]:
len(df.Heating[df.Heating=='GasA']) / len(df)

0.9780821917808219

## HeatingQC

*Heating Quality and Condition*

In [536]:
df.HeatingQC.isna().sum()

0

In [537]:
px.bar(df.HeatingQC.value_counts())

## Central Air

In [538]:
df.CentralAir.value_counts()

Y    1365
N      95
Name: CentralAir, dtype: int64

## Electrical

- SBrkr -> Standard Circuit Breakers & Romex
- FuseA -> Fuse Box over 60 AMP and all Romex wiring (Average)
- FuseF -> 60 AMP Fuse Box and mostly Romex wiring (Fair)
- FuseP -> 60 AMP Fuse Box and mostly knob & tube wiring (poor)
- Mix -> Mixed

* interesting: while fuse boxes and circuit breakers perform the same job, a fuse box melts its fuse to break the circuit, require replacement each time, whereas a circuit breaker "trips" and disconnects (and can be turned on simply by switching)

In [539]:
df.Electrical.value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [540]:
len(df.Electrical[df.Electrical=='SBrkr']) / len(df)

0.9136986301369863

## 1stFlrSF

*First floor sqft*

In [541]:
df['1stFlrSF'].isna().sum()

0

In [542]:
px.histogram(df['1stFlrSF'])

## 2ndFlrSF

*Second Floor SqFt*

In [543]:
df['2ndFlrSF'].isna().sum()

0

In [544]:
px.histogram(df['2ndFlrSF'])

## LowQualFinSF

*Low quality finished square feet (all floors)*

I have no idea what the above means

In [545]:
df.LowQualFinSF.isna().sum()

0

In [546]:
len(df.LowQualFinSF[df.LowQualFinSF>0]) / len(df)

0.01780821917808219

In [547]:
df.LowQualFinSF.describe()

count    1460.000000
mean        5.844521
std        48.623081
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       572.000000
Name: LowQualFinSF, dtype: float64

In [548]:
px.histogram(df.LowQualFinSF)

## GrLivArea

*Above grade (ground) living area square feet*

In [549]:
df.GrLivArea.isna().sum()

0

In [550]:
px.histogram(df.GrLivArea)

## FullBath

*Count of full bathrooms above ground*

In [551]:
df.FullBath.isna().sum()

0

In [552]:
px.bar(df.FullBath.value_counts())

... how are there 9 records with 0 full bathrooms? Basement bathrooms?

In [553]:
df.loc[(df.FullBath==0), ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath']]

,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath
53,2,0,0,1
188,2,0,0,2
375,1,0,0,1
597,0,2,0,2
634,2,0,0,0
916,1,0,0,0
1163,2,0,0,2
1213,1,1,0,0
1270,2,0,0,1


## HalfBath

*Count of half bathrooms*

In [554]:
df.HalfBath.isna().sum()

0

In [555]:
px.bar(df.HalfBath.value_counts())

## Bedroom

*Count bedrooms above ground*

Notes state that this count doesn't include basement bedrooms, though there is no other column that does so ¯\\_(ツ)_/¯

In [556]:
df.BedroomAbvGr.isna().sum()

0

In [557]:
px.bar(df.BedroomAbvGr.value_counts())

## TotRmsAbvGrd

*Total rooms above grade (does not include bathrooms)*

In [558]:
df.TotRmsAbvGrd.isna().sum()

0

In [559]:
px.bar(df.TotRmsAbvGrd.value_counts())

In [560]:
df.TotRmsAbvGrd.describe()

count    1460.000000
mean        6.517808
std         1.625393
min         2.000000
25%         5.000000
50%         6.000000
75%         7.000000
max        14.000000
Name: TotRmsAbvGrd, dtype: float64

## Kitchen

*Kitchens above grade*

Soooo no basement kitchens

In [561]:
df.KitchenAbvGr.isna().sum()

0

In [562]:
px.bar(df.KitchenAbvGr.value_counts())

## KitchenQual

*Kitchen quality*

In [563]:
px.bar(df.KitchenQual.replace(qualitative_rating_map).value_counts())

What about those fancy 2 kitchen houses? Passover kitchens??

In [564]:
px.bar(df.KitchenQual[df.KitchenAbvGr>1].replace(qualitative_rating_map).value_counts(), title='Multi-kitchen qualities')

## Functional

*Home functionality (Assume typical unless deductions are warranted)*

I have no idea what the above decription means and this col name is the most ambiguous one yet.

In [565]:
df.Functional.isna().sum()

0

In [566]:
col_map = {
    "Typ" : "Typical Functionality",
    "Min1": "Minor Deductions 1",
    "Min2": "Minor Deductions 2",
    "Mod" : "Moderate Deductions",
    "Maj1": "Major Deductions 1",
    "Maj2": "Major Deductions 2",
    "Sev" : "Severely Damaged",
    "Sal" : "Salvage only",
}

In [567]:
px.bar(df.Functional.replace(col_map).value_counts())

In [568]:
len(df.Functional[df.Functional=="Typ"])/len(df)

0.9315068493150684

## Fireplaces

*Count of fireplaces*

In [569]:
df.Fireplaces.isna().sum()

0

In [570]:
px.bar(df.Fireplaces.value_counts())

## FireplaceQu

*Fireplace quality*

Possible values:
- Excellent - Exceptional Masonry Fireplace
- Good - Masonry Fireplace in main level
- Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
- Fair - Prefabricated Fireplace in basement
- Poor - Ben Franklin Stove
- No Fireplace

In [571]:
df.FireplaceQu.isna().sum()

690

In [572]:
px.bar(df.FireplaceQu.replace(qualitative_rating_map).value_counts())

## GarageType

*Garage Location*

In [573]:
df.GarageType.isna().sum()

81

In [574]:
col_map = {
    "2Types" : "More than one type of garage",
    "Attchd" : "Attached to home",
    "Basment": "Basement Garage",
    "BuiltIn": "Built-In (Garage part of house - typically has room above garage)",
    "CarPort": "Car Port",
    "Detchd" : "Detached from home",
    "NA"     : "No Garage",
}

In [575]:
px.bar(df.GarageType.replace(col_map).value_counts())

## GarageYrBlt

In [576]:
df.GarageYrBlt.isna().sum()

81

In [577]:
px.histogram(df.GarageYrBlt)

How often were garages built afterward?

In [578]:
(df.YearBuilt < df.GarageYrBlt).sum()

281

In [579]:
(df.YearRemodAdd < df.GarageYrBlt).sum()

127

In [580]:
(df.YearBuilt > df.GarageYrBlt).sum()

9

????

In [581]:
df.loc[(df.YearBuilt > df.GarageYrBlt), ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'GarageType']].T

,29,93,324,600,736,1103,1376,1414,1418
YearBuilt,1927,1910,1967,2005,1950,1959,1930,1923,1963
YearRemodAdd,1950,1998,2007,2005,1950,1959,1950,2000,1963
GarageYrBlt,1920,1900,1961,2003,1949,1954,1925,1922,1962
GarageType,Detchd,Detchd,BuiltIn,BuiltIn,Detchd,BuiltIn,Detchd,Detchd,Detchd


While I'm puzzled by how a builtin garage can be older than the house it is built into, this shouldn't actually affect the prediction algorithm much.

## GarageFinish

In [582]:
df.GarageFinish.isna().sum()

81

In [583]:
col_map = {
    "Fin": "Finished",
    "RFn": "Rough Finished",
    "Unf": "Unfinished",
    "NA": "No Garage",
}

In [584]:
px.bar(df.GarageFinish.replace(col_map).value_counts())

## GarageCars

In [585]:
df.GarageCars.isna().sum()

0

zero is used here in place of NA

In [586]:
px.bar(df.GarageCars.value_counts())

## GarageArea

In [587]:
df.GarageArea.isna().sum()

0

In [588]:
px.histogram(df.GarageArea)

In [589]:
px.histogram(df.GarageArea / df.GarageCars, title='Garage Area per Car')

## GarageQual

In [590]:
df.GarageQual.isna().sum()

81

In [591]:
px.bar(df.GarageQual.replace(qualitative_rating_map).value_counts())

## GarageCond

In [592]:
df.GarageCond.isna().sum()

81

In [593]:
px.bar(df.GarageCond.replace(qualitative_rating_map).value_counts())

## PavedDrive

*Is driveway paved?*

In [594]:
df.PavedDrive.isna().sum()

0

In [595]:
px.bar(df.PavedDrive.value_counts())

## WoodDeckSF

In [596]:
df.WoodDeckSF.isna().sum()

0

In [597]:
px.histogram(df.WoodDeckSF)

## OpenPorchSF

In [598]:
df.OpenPorchSF.isna().sum()

0

In [599]:
px.histogram(df.OpenPorchSF)

## EnclosedPorch

In [600]:
df.EnclosedPorch.isna().sum()

0

In [601]:
px.histogram(df.EnclosedPorch)

In [602]:
len(df.EnclosedPorch[df.EnclosedPorch>0])/len(df)

0.14246575342465753

## 3SsnPorch

What is a 3 season porch?

Well, it's a mostly enclosed/screened porch which is good for all but Winter

In [603]:
df['3SsnPorch'].isna().sum()

0

In [604]:
px.histogram(df['3SsnPorch'])

In [605]:
len(df['3SsnPorch'][df['3SsnPorch']>0])/len(df)

0.01643835616438356

## ScreenPorch

In [606]:
df.ScreenPorch.isna().sum()

0

In [607]:
px.histogram(df.ScreenPorch)

In [608]:
len(df.ScreenPorch[df.ScreenPorch>0])/len(df)

0.07945205479452055

In [609]:
pct_openporch

0.5506849315068493

In [610]:
pct_openporch = len(df.OpenPorchSF[df.OpenPorchSF>0])/len(df)
pct_enclosedporch = len(df.EnclosedPorch[df.EnclosedPorch>0])/len(df)
pct_3sporch = len(df['3SsnPorch'][df['3SsnPorch']>0])/len(df)
pct_screen = len(df.ScreenPorch[df.ScreenPorch>0])/len(df)
pct_none = 1 - pct_openporch - pct_enclosedporch - pct_3sporch - pct_screen

px.bar(x=['Open', 'Enclosed', '3 Season', 'Screen', 'None'], y=[pct_openporch, pct_enclosedporch, pct_3sporch, pct_screen, pct_none])


## PoolArea

In [611]:
df.PoolArea.isna().sum()

0

In [612]:
px.histogram(df.PoolArea)

In [613]:
len(df[df.PoolArea>0]) / len(df)

0.004794520547945206

## PoolQC

*Pool Quality*

But there are barely any pools...

In [614]:
df.PoolQC.isna().sum()

1453

In [615]:
# Just out of curiosity
px.bar(df.PoolQC.replace(qualitative_rating_map).value_counts())

## Fence

*Fence Quality*

In [616]:
df.Fence.isna().sum()

1179

In [617]:
df.Fence.isna().sum() / len(df)

0.8075342465753425

In [618]:
col_map = {
    "GdPrv": "Good Privacy",
    "MnPrv": "Minimum Privacy",
    "GdWo" : "Good Wood",
    "MnWw" : "Minimum Wood/Wire",
    "NA"   : "No Fence",
}

In [619]:
px.bar(df.Fence.replace(col_map).value_counts())

## MiscFeature

*Miscellaneous feature not covered in other categories*

Oh...

In [620]:
df.MiscFeature.isna().sum()

1406

In [621]:
df.MiscFeature.isna().sum() / len(df)

0.963013698630137

In [622]:
col_map = {
    "Elev": "Elevator",
    "Gar2": "2nd Garage (if not described in garage section)",
    "Othr": "Other",
    "Shed": "Shed (over 100 SF)",
    "TenC": "Tennis Court",
    "NA": "None",
}

In [623]:
px.bar(df.MiscFeature.replace(col_map).value_counts())

Soo many questions - like what is going on in those "Large Sheds" and who owns that tennis court??

In [624]:
df[df.MiscFeature=='TenC']

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1386,1387,60,RL,80.0,16692,Pave,NaN,IR1,Lvl,AllPub,...,519,Fa,MnPrv,TenC,2000,7,2006,WD,Normal,250000


## MiscVal

*$Value of miscellaneous feature*

I suppose to lower sale price by?

In [625]:
df.MiscVal.isna().sum()

0

In [626]:
px.histogram(df.MiscVal[~df.MiscFeature.isna()])

## MoSold

*Month sold*

In [627]:
df.MoSold.isna().sum()

0

In [628]:
px.bar(df.MoSold.value_counts())

## YrSold

*Year Sold*

In [629]:
df.YrSold.isna().sum()

0

In [630]:
px.bar(df.YrSold.value_counts())

## SaleType

In [631]:
df.SaleType.isna().sum()

0

In [632]:
col_map = {
    "WD"   : "Warranty Deed - Conventional",
    "CWD"  : "Warranty Deed - Cash",
    "VWD"  : "Warranty Deed - VA Loan",
    "New"  : "Home just constructed and sold",
    "COD"  : "Court Officer Deed/Estate",
    "Con"  : "Contract 15% Down payment regular terms",
    "ConLw": "Contract Low Down payment and low interest",
    "ConLI": "Contract Low Interest",
    "ConLD": "Contract Low Down",
    "Oth"  : "Other",
}

In [633]:
df.SaleType.replace(col_map).value_counts()

Warranty Deed - Conventional                  1267
Home just constructed and sold                 122
Court Officer Deed/Estate                       43
Contract Low Down                                9
Contract Low Interest                            5
Contract Low Down payment and low interest       5
Warranty Deed - Cash                             4
Other                                            3
Contract 15% Down payment regular terms          2
Name: SaleType, dtype: int64

In [634]:
px.bar(df.SaleType.replace(col_map).value_counts())

## SaleCondition

*Condition of Sale*

- Normal -> Normal Sale
- Abnorml -> Abnormal Sale -  trade, foreclosure, short sale
- AdjLand -> Adjoining Land Purchase
- Alloca -> Allocation - two linked properties with separate deeds, typically condo with a garage unit
- Family -> Sale between family members
- Partial -> Home was not completed when last assessed (associated with New Homes)


In [635]:
df.SaleCondition.isna().sum()

0

In [636]:
px.bar(df.SaleCondition.replace(col_map).value_counts())

## SalePrice

*This is a target value*

Obviously we don't want to touch these values, however out of curiosity...

In [637]:
df.SalePrice.isna().sum()

0

Oh good

In [638]:
px.histogram(df.SalePrice)

In [639]:
px.histogram(np.log(df.SalePrice))

# Multiple Col Analysis

In [640]:
# Month/Year Median Sales Price
base_year = df.YrSold.min()
df['month_id'] = (df.YrSold - base_year) * 12 + df.MoSold

# The final month seems to have lower sales and might not be a complete month
df = df[df.month_id < df.month_id.max()]

# Group by Month ID
g = df[['month_id', 'SalePrice']].groupby(by='month_id').agg(['mean', 'median', 'count'])
g.columns = g.columns.droplevel()


In [641]:
mean_ols = list(px.scatter(g['mean'], trendline="ols").select_traces())[1].y

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=list(g.index), y=g['mean'], name="Mean Price"), secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=list(g.index), y=mean_ols, name="OLS"), secondary_y=True,
)
fig.add_trace(
    go.Bar(x=list(g.index), y=g['count'], opacity=0.2, name="Volume")
)
fig.add_trace(
    go.Scatter(x=list(g.index), y=g['median'], name="Median Price"), secondary_y=True,
)

fig.update_layout(title_text="Sales Price and Volume Over Time")
fig.show()

# Handling Null Values

In [642]:
for col in df.columns:
    count_na = df[col].isna().sum()
    pct_na = count_na / len(df)
    if pct_na > 0.01:
        print(f'{col} contains {pct_na*100:.2f}% NA values')

LotFrontage contains 17.74% NA values
Alley contains 93.74% NA values
BsmtQual contains 2.48% NA values
BsmtCond contains 2.48% NA values
BsmtExposure contains 2.54% NA values
BsmtFinType1 contains 2.48% NA values
BsmtFinType2 contains 2.54% NA values
FireplaceQu contains 47.32% NA values
GarageType contains 5.50% NA values
GarageYrBlt contains 5.50% NA values
GarageFinish contains 5.50% NA values
GarageQual contains 5.50% NA values
GarageCond contains 5.50% NA values
PoolQC contains 99.52% NA values
Fence contains 80.74% NA values
MiscFeature contains 96.35% NA values
